In [95]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pylab as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from scipy.stats import f, chi2
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn import tree

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [81]:
df=pd.read_csv("digit-recognizer/train.csv")
df_train=df.drop("label",axis=1)
df_y_train=df["label"]

In [82]:
df.shape

(42000, 785)

### Apply 9-dimensional PCA – Call it the Digits-PCA dataset

In [76]:
pcs = PCA(n_components=9)
xpca=pcs.fit_transform(df_train)

In [77]:
digits_PCA=pd.DataFrame(xpca)
digits_PCA

,0,1,2,3,4,5,6,7,8
0,-661.595782,-699.311292,183.283200,120.606776,-81.083187,489.433576,-683.534785,85.352545,348.381971
1,1701.451728,-360.550718,-501.806468,335.428255,-442.374192,738.431186,653.841397,-176.502322,-7.523864
2,-886.894452,-293.766037,67.155166,78.265585,-473.715753,-323.531439,437.841240,-305.263104,-195.043997
3,-165.755624,300.182295,-64.144655,759.703923,-425.851411,157.384689,-304.071222,276.425373,-45.669436
4,1923.709762,-449.152186,-548.614075,188.559297,-651.729360,990.081006,564.458092,-255.783508,124.767582
...,...,...,...,...,...,...,...,...,...
41995,760.355343,-249.853171,-406.992770,-322.859372,-1005.132592,556.400195,-272.160977,-632.824418,-164.663918
41996,-1059.873238,-302.293491,2.941449,38.665850,-360.059931,-150.787627,330.415625,-211.579129,118.637211
41997,336.270023,950.016029,-168.605945,-692.676680,-539.357226,29.211774,500.215335,653.407324,86.462913
41998,327.399371,62.015601,697.535585,482.813474,-154.816286,-897.963779,402.292098,-114.204076,-163.635519


### Apply 9-dimensional FISHER – Call it the Digits-FISHER dataset

In [64]:
lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True, n_components=9)
X_lda=lda.fit_transform(df_train, df_y_train)

In [75]:
Digits_FISHER=pd.DataFrame(X_lda)
Digits_FISHER

,0,1,2,3,4,5,6,7,8
0,0.706982,3.702191,-0.546160,1.083590,-1.282057,-0.640238,-0.161646,0.711746,0.098052
1,-4.753373,-3.257093,-2.983682,-1.244001,-1.880934,-0.898564,0.114414,-1.097409,1.235790
2,0.426475,5.168707,-0.215028,0.248895,-3.737808,0.168903,0.546867,0.164058,-0.314798
3,-0.978410,-0.555503,1.147945,-0.324528,-0.997568,-0.858390,0.979497,1.846044,-0.207963
4,-4.878184,-3.244367,-4.723876,-0.850046,-1.923177,-2.093587,0.166724,-2.228554,0.999328
...,...,...,...,...,...,...,...,...,...
41995,-2.846328,-1.655570,-3.661467,0.658924,-1.528190,-0.680945,0.511808,-1.606290,-1.122931
41996,1.776876,4.591485,-0.370188,0.200660,-2.247352,-0.005172,1.093822,-0.204246,-0.427021
41997,3.078790,-2.807856,-3.082087,-2.657304,-1.939725,2.647739,-2.012444,0.094553,-0.028155
41998,-2.901593,-1.216585,4.178510,-1.426573,-1.557683,2.061918,1.035576,0.932003,-0.135465


### Divide the data into 65% training and 35% test (after projection)

In [85]:
Xpca=digits_PCA
Xlda=Digits_FISHER
y=df_y_train
Xpca_train, Xpca_test, ypca_train, ypca_test = train_test_split(Xpca, y, test_size=0.35, random_state=42)
Xlda_train, Xlda_test, ylda_train, ylda_test = train_test_split(Xlda, y, test_size=0.35, random_state=42)


### Build a Decision Tree classifier with depth 5, purity threshold 0.8 for each dataset
o What is the train and test accuracy of Digits-PCA-Tree classifier?

In [92]:

        data={"method":"PCA"}
        data["Maximum Depth"]=5
        data["Purity Threshold"]=0.8

        classifier = DecisionTreeClassifier(criterion='entropy',
            splitter='best',
            max_depth=5,
            min_impurity_decrease=1-0.8  
        )
        #since for impurity values greater than 0.2, the accuracy value is same for all depths 
        #so, I have taken (1-impurity) to have some variance in accuracy value
        classifier.fit(Xpca_train, ypca_train)

        #train accuracy
        ypca_pred_train = classifier.predict(Xpca_train)
        accuracy_train = accuracy_score(ypca_train, ypca_pred_train)
        data["accuracy on train set"]=accuracy_train

        train_mse = mean_squared_error(ypca_train, ypca_pred_train)
        data["MSE on train set"]=train_mse

        #test accuracy
        ypca_pred = classifier.predict(Xpca_test)
        accuracy = accuracy_score(ypca_test, ypca_pred)
        data["accuracy on test set"]=accuracy

        test_mse = mean_squared_error(ypca_test, ypca_pred)
        data["MSE on test set"]=test_mse
        #output.append(data)
        pd.DataFrame([data])

,method,Maximum Depth,Purity Threshold,accuracy on train set,MSE on train set,accuracy on test set,MSE on test set
0,PCA,5,0.8,0.294396,6.859963,0.296667,6.839728


### Build a Decision Tree classifier with depth 5, purity threshold 0.8 for each dataset
o What is the train and test accuracy of Digits-FISHER-Tree classifier?

In [93]:

data={"method":"Fisher"}
data["Maximum Depth"]=5
data["Purity Threshold"]=0.8

classifier = DecisionTreeClassifier(criterion='entropy',
    splitter='best',
    max_depth=5,
    min_impurity_decrease=1-0.8  
)
#since for impurity values greater than 0.2, the accuracy value is same for all depths 
#so, I have taken (1-impurity) to have some variance in accuracy value
classifier.fit(Xlda_train, ylda_train)

#train accuracy
ylda_pred_train = classifier.predict(Xlda_train)
accuracy_train = accuracy_score(ylda_train, ylda_pred_train)
data["accuracy on train set"]=accuracy_train

train_mse = mean_squared_error(ylda_train, ylda_pred_train)
data["MSE on train set"]=train_mse

#test accuracy
ylda_pred = classifier.predict(Xlda_test)
accuracy = accuracy_score(ylda_test, ylda_pred)
data["accuracy on test set"]=accuracy

test_mse = mean_squared_error(ylda_test, ylda_pred)
data["MSE on test set"]=test_mse
#output.append(data)
pd.DataFrame([data])

,method,Maximum Depth,Purity Threshold,accuracy on train set,MSE on train set,accuracy on test set,MSE on test set
0,Fisher,5,0.8,0.464029,5.40293,0.46551,5.48898


### Build a k-NN classifier with k = 5 on both datasets
o What is the train and test accuracy of Digits-FISHER-kNN classifier?

In [97]:
acc={"method":"Fisher"}
i=5
knn = KNeighborsClassifier(n_neighbors=i)
acc["k"]=i
# Train the classifier on the training data
knn.fit(Xlda_train, ylda_train)

y_pred_t = knn.predict(Xlda_train)

train_accuracy = accuracy_score(ylda_train, y_pred_t)
acc["Train Accuracy"]=train_accuracy
# Make predictions on the test data
y_pred = knn.predict(Xlda_test)
# Calculate the accuracy of the model
test_accuracy = accuracy_score(ylda_test, y_pred)
#print(f"Accuracy: {accuracy:.2f}")
acc["Test Accuracy"]=test_accuracy
pd.DataFrame([acc])


,method,k,Train Accuracy,Test Accuracy
0,Fisher,5,0.936557,0.918163


### Build a k-NN classifier with k = 5 on both datasets
o What is the train and test accuracy of Digits-PCA-kNN classifier?

In [101]:
acc={"method":"PCA"}
i=5
knn = KNeighborsClassifier(n_neighbors=i)
acc["k"]=i
# Train the classifier on the training data
knn.fit(Xpca_train, ypca_train)

y_pred_t = knn.predict(Xpca_train)

train_accuracy = accuracy_score(ypca_train, y_pred_t)
acc["Train Accuracy"]=train_accuracy
# Make predictions on the test data
y_pred = knn.predict(Xpca_test)
# Calculate the accuracy of the model
test_accuracy = accuracy_score(ypca_test, y_pred)
#print(f"Accuracy: {accuracy:.2f}")
acc["Test Accuracy"]=test_accuracy
pd.DataFrame([acc])

,method,k,Train Accuracy,Test Accuracy
0,PCA,5,0.936227,0.906054


### Build a Logistic Regression with between any pair of classes (1 vs 7)

In [109]:
pca_lr_df=pd.concat([df_y_train,digits_PCA], axis=1)
pca_lr_df=pca_lr_df[(pca_lr_df["label"]==1) | (pca_lr_df["label"]==7)]

fisher_lr_df=pd.concat([df_y_train,Digits_FISHER], axis=1)
fisher_lr_df=fisher_lr_df[(fisher_lr_df["label"]==1) | (fisher_lr_df["label"]==7)]

Xpca=pca_lr_df.drop('label',axis=1)
Xlda=fisher_lr_df.drop(['label'],axis=1)
ypca=pca_lr_df['label']
ylda=fisher_lr_df['label']
Xpca_train, Xpca_test, ypca_train, ypca_test = train_test_split(Xpca, ypca, test_size=0.35, random_state=42)
Xlda_train, Xlda_test, ylda_train, ylda_test = train_test_split(Xlda, ylda, test_size=0.35, random_state=42)

o What is the train and test accuracy of Digits-PCA-Logistic-Regression classifier?

In [113]:
acc={"method":"PCA"}
model = LogisticRegression()
result=model.fit(Xpca_train, ypca_train)

# Make predictions on the test set
y_pred = model.predict(Xpca_test)
y_pred_t = model.predict(Xpca_train)

# Calculate the accuracy of the model
accuracy = accuracy_score(ypca_test, y_pred)
train_accuracy = accuracy_score(ypca_train, y_pred_t)
acc["test Accuracy"]= accuracy
acc["train Accuracy"]= train_accuracy

pd.DataFrame([acc])

,method,test Accuracy,train Accuracy
0,PCA,0.987736,0.991025


o What is the train and test accuracy of Digits-FISHER-Logistic-Regression classifier?

In [114]:
acc={"method":"Fisher"}
model = LogisticRegression()
result=model.fit(Xlda_train, ylda_train)

# Make predictions on the test set
y_pred = model.predict(Xlda_test)
y_pred_t = model.predict(Xlda_train)

# Calculate the accuracy of the model
accuracy = accuracy_score(ylda_test, y_pred)
train_accuracy = accuracy_score(ylda_train, y_pred_t)
acc["test Accuracy"]= accuracy
acc["train Accuracy"]= train_accuracy

pd.DataFrame([acc])

,method,test Accuracy,train Accuracy
0,Fisher,0.987421,0.98967


### Comment on whether PCA vs. FISHER is better for projection

From the above analysis, it is visible that Fisher is better for projection when used for supervised learning.